In [1]:
import plotly.express as px
import pandas as pd

# Data Creation

Next Gen Stats defines average separation as the yards between a wide receiver or tight end and the closest defender at the catch or incompletion. So, it's the yardage distance between the pass-catcher and defender when the player attempts to make the reception. From this [article](https://www.rotoballer.com/next-gen-stats-review-2023-wide-receivers-and-tight-ends/1334129).

## Source Data

In [2]:
df = pd.read_csv("./df_ngs_wr_2020_2023.csv")
df_ngs = df.copy().query("week != 0 and season_type == 'REG'")

In [3]:
df_ngs.head()

,season,season_type,week,player_display_name,player_position,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,...,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name
132,2020,REG,1,Curtis Samuel,WR,CAR,8.771667,3.172406,7.430000,21.024335,...,38.0,0,1.068000,2.811129,-1.743129,00-0033282,Curtis,Samuel,10,C.Samuel
133,2020,REG,1,Amari Cooper,WR,DAL,8.686429,2.408512,7.993571,42.671395,...,81.0,0,1.675000,2.228499,-0.553499,00-0031544,Amari,Cooper,19,A.Cooper
134,2020,REG,1,DeSean Jackson,WR,PHI,8.604286,2.955064,29.038571,40.384233,...,46.0,0,3.950000,3.576771,0.373229,00-0026189,DeSean,Jackson,10,D.Jackson
135,2020,REG,1,Henry Ruggs III,WR,LV,8.178000,4.694896,18.524000,60.484556,...,55.0,0,7.630000,8.236114,-0.606114,00-0036357,Henry,Ruggs,11,H.Ruggs
136,2020,REG,1,Darren Waller,TE,LV,8.081667,3.208908,3.361250,17.560243,...,45.0,0,5.728333,6.115772,-0.387439,00-0031610,Darren,Waller,83,D.Waller


In [4]:
df_ngs.week.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18])

In [5]:
df_ngs[df_ngs.week == 0].head()

,season,season_type,week,player_display_name,player_position,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,...,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_jersey_number,player_short_name


# Getting Aggregates from NGS Data

In [6]:
df_ngs.columns

Index(['season', 'season_type', 'week', 'player_display_name',
       'player_position', 'team_abbr', 'avg_cushion', 'avg_separation',
       'avg_intended_air_yards', 'percent_share_of_intended_air_yards',
       'receptions', 'targets', 'catch_percentage', 'yards', 'rec_touchdowns',
       'avg_yac', 'avg_expected_yac', 'avg_yac_above_expectation',
       'player_gsis_id', 'player_first_name', 'player_last_name',
       'player_jersey_number', 'player_short_name'],
      dtype='object')

In [7]:
df_ngs["fantasy_points"] = df_ngs["yards"] * .10 + df_ngs["rec_touchdowns"] * 6
df_ngs["fantasy_points_ppr"] = df_ngs["yards"] * .10 + df_ngs["rec_touchdowns"] * 6 + df_ngs["receptions"]

In [8]:
df_wr = df_ngs.groupby(["season", "player_gsis_id"]).agg({
    'player_display_name': lambda x: x.mode()[0],
    'avg_separation': ["mean", "min", "max", "median"],
    'avg_cushion': ["mean", "min", "max", "median"],
    "avg_intended_air_yards": ["mean", "min", "max", "median"],
    "percent_share_of_intended_air_yards": ["mean", "min", "max", "median"],
    "catch_percentage": ["mean", "min", "max", "median"],
    "targets": ["sum", "mean", "min", "max", "median"],
    "receptions": ["sum", "mean", "min", "max", "median"],
    "yards": ["sum", "mean", "min", "max", "median"],
    "rec_touchdowns": ["sum", "mean", "min", "max", "median"],
    "fantasy_points": ["sum", "mean", "min", "max", "median"],
    "fantasy_points_ppr": ["sum", "mean", "min", "max", "median"]
})

In [9]:
df_wr.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_wr.columns]

In [10]:
df_wr

player_display_name_<lambda>  avg_separation_mean  \
season player_gsis_id                                                     
2020   00-0022127                     Jason Witten             2.234854   
       00-0022921                 Larry Fitzgerald             3.440320   
       00-0025418                       Greg Olsen             2.814102   
       00-0026035                   Danny Amendola             3.121636   
       00-0026189                   DeSean Jackson             3.289270   
...                                            ...                  ...   
2023   00-0039067                      Rashee Rice             4.226126   
       00-0039074                      Davis Allen             5.439141   
       00-0039075                       Puka Nacua             2.813624   
       00-0039144                    Luke Musgrave             4.179088   
       00-0039146                      Jayden Reed             3.456029   

                       avg_separation_min  avg_separation_max  \
season player_gsis_id                                           
2020   00-0022127                2.234854            2.234854   
       00-0022921                2.284457            4.534797   
       00-0025418                1.762624            3.865581   
       00-0026035                1.990407            5.121982   
       00-0026189                2.955064            3.529793   
...                                   ...                 ...   
2023   00-0039067                2.715172            6.029107   
       00-0039074                5.439141            5.439141   
       00-0039075                1.637036            3.715585   
       00-0039144                3.229740            5.417008   
       00-0039146                2.016319            5.141321   

                       avg_separation_median  avg_cushion_mean  \
season player_gsis_id                                            
2020   00-0022127                   2.234854          4.460000   
       00-0022921                   3.600421          6.143298   
       00-0025418                   2.814102          6.368429   
       00-0026035                   2.877260          6.483000   
       00-0026189                   3.382953          7.815058   
...                                      ...               ...   
2023   00-0039067                   4.276784          6.287451   
       00-0039074                   5.439141          2.640000   
       00-0039075                   2.936664          6.005909   
       00-0039144                   4.034803          5.668125   
       00-0039146                   3.307238          6.668942   

                       avg_cushion_min  avg_cushion_max  avg_cushion_median  \
season player_gsis_id                                                         
2020   00-0022127             4.460000         4.460000            4.460000   
       00-0022921             4.844000         6.982857            6.478929   
       00-0025418             3.842857         8.894000            6.368429   
       00-0026035             4.566667         8.118333            6.430000   
       00-0026189             5.542000         9.298889            8.604286   
...                                ...              ...                 ...   
2023   00-0039067             4.802727         8.074286            5.987500   
       00-0039074             2.640000         2.640000            2.640000   
       00-0039075             4.193333         8.182857            5.974500   
       00-0039144             3.584000         6.920000            6.084250   
       00-0039146             3.774286         9.366667            6.400125   

                       avg_intended_air_yards_mean  ...  fantasy_points_sum  \
season player_gsis_id                               ...                       
2020   00-0022127                         6.358000  ...                 1.2   
       00-0022921                         5.321955  ...                33.8 

In [15]:
df_wr.rename(columns={"player_display_name_<lambda>": "player_name"}, inplace=True)

In [20]:
df_wr.query("player_name=='Nico Collins'")

,,player_name,avg_separation_mean,avg_separation_min,avg_separation_max,avg_separation_median,avg_cushion_mean,avg_cushion_min,avg_cushion_max,avg_cushion_median,avg_intended_air_yards_mean,...,fantasy_points_sum,fantasy_points_mean,fantasy_points_min,fantasy_points_max,fantasy_points_median,fantasy_points_ppr_sum,fantasy_points_ppr_mean,fantasy_points_ppr_min,fantasy_points_ppr_max,fantasy_points_ppr_median
season,player_gsis_id,,,,,,,,,,,,,,,,,,,,,
2021,00-0036554,Nico Collins,2.249637,1.473149,2.755607,2.412512,4.749833,2.855000,7.158000,4.760000,13.080198,...,27.1,4.516667,2.8,6.9,3.95,45.1,7.516667,4.8,11.9,6.95
2022,00-0036554,Nico Collins,2.992058,1.877913,4.637888,2.651033,6.032467,4.490000,7.791000,6.025714,12.605152,...,50.1,7.157143,4.4,10.9,6.50,80.1,11.442857,9.8,15.9,10.50
2023,00-0036554,Nico Collins,2.961274,1.535008,4.975199,2.865368,5.904355,3.832222,8.533333,5.727917,11.422473,...,169.1,14.091667,3.0,28.8,9.70,243.1,20.258333,7.0,35.8,14.70
